# Practical Exam: Hotel Operations

LuxurStay Hotels is a major, international chain of hotels. They offer hotels for both business and leisure travellers in major cities across the world. The chain prides themselves on the level of customer service that they offer. 

However, the management has been receiving complaints about slow room service in some hotel branches. As these complaints are impacting the customer satisfaction rates, it has become a serious issue. Recent data shows that customer satisfaction has dropped from the 4.5 rating that they expect. 

You are working with the Head of Operations to identify possible causes and hotel branches with the worst problems. 

## Data

The following schema diagram shows the tables available. You have only been provided with data where customers provided a feedback rating.

![hotel_operations](hotel_operations.png)

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `branch` table, and the data team have provided the following data description. 

Write a query to return data matching this description, including identifying and cleaning all invalid values. You must match all column names and description criteria. Your output should be a DataFrame named 'clean_branch_data'.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|id | Nominal. The unique identifier of the hotel. </br>Missing values are not possible due to the database structure.|
| location | Nominal. The location of the particular hotel. One of four possible values, 'EMEA', 'NA', 'LATAM' and 'APAC'. </br>Missing values should be replaced with “Unknown”. |
| total_rooms | Discrete. The total number of rooms in the hotel. Must be a positive integer between 1 and 400. </br>Missing values should be replaced with the default number of rooms, 100. |
| staff_count | Discrete. The number of staff employeed in the hotel service department. </br>Missing values should be replaced with the total_rooms multiplied by 1.5. |
| opening_date | Discrete. The year in which the hotel opened. This can be any value between 2000 and 2023. </br>Missing values should be replaced with 2023. |
| target_guests | Nominal. The primary type of guest that is expected to use the hotel. Can be one of 'Leisure' or 'Business'. </br>Missing values should be replaced with 'Leisure'. |

In [3]:
-- Write your query for task 1 in this cell
SELECT DISTINCT id, location,
	CASE WHEN total_rooms IS NULL THEN 100 ELSE total_rooms END AS total_rooms,
	staff_count,
	REPLACE (public.branch.opening_date, '-', '2023') AS opening_date,
	CASE WHEN target_guests ILIKE 'B%' THEN 'Business'
	WHEN target_guests ILIKE 'L%' THEN 'Leisure'
	ElSE target_guests END AS target_guests
FROM public.branch


,target_guests
0,Business
1,Leisure
2,Leisure
3,Business
4,Business
...,...
95,Business
96,Business
97,Leisure
98,Business


# Task 2

The Head of Operations wants to know whether there is a difference in time taken to respond to a customer request in each hotel. They already know that different services take different lengths of time. 

Calculate the average and maximum duration for each branch and service. 
- Your output should be a DataFrame named 'average_time_service'
- It should include the columns `service_id`, `branch_id`, `avg_time_taken` and `max_time_taken`
- Values should be rounded to two decimal places where appropriate. 

In [8]:
-- Write your query for task 2 in this cell
SELECT service_id, branch_id, ROUND (AVG (time_taken), 2) AS avg_time_taken, ROUND (MAX (time_taken), 2) AS max_time_taken
FROM public.request
GROUP BY service_id, branch_id
ORDER BY service_id, branch_id

,service_id,branch_id,avg_time_taken,max_time_taken
0,1,1,2.44,12.0
1,1,2,2.14,6.0
2,1,3,2.28,6.0
3,1,4,2.18,9.0
4,1,5,2.52,10.0
...,...,...,...,...
380,4,96,9.79,26.0
381,4,97,9.59,21.0
382,4,98,9.14,14.0
383,4,99,9.13,13.0


# Task 3

The management team want to target improvements in `Meal` and `Laundry` service in Europe (`EMEA`) and Latin America (`LATAM`). 

Write a query to return the `description` of the service, the `id` and `location` of the branch, the id of the request as `request_id` and the `rating` for the services and locations of interest to the management team. 

Your output should be a DataFrame named 'target_hotels'.

Use the original branch table, not the output of task 1. 

In [9]:
-- Write your query for task 3 in this cell
SELECT DISTINCT s.description, b.id, b.location, r.id AS request_id, r.rating
FROM public.request AS r
INNER JOIN public.service AS s
ON s.id = r.service_id
INNER JOIN public.branch AS b
ON b.id = r.branch_id
WHERE s.description IN ('Meal', 'Laundry')
AND b.location IN ('EMEA', 'LATAM')
ORDER BY description, location

,description,id,location,request_id,rating
0,Laundry,7,EMEA,2462,3
1,Laundry,7,EMEA,2921,4
2,Laundry,7,EMEA,2945,3
3,Laundry,7,EMEA,4557,4
4,Laundry,7,EMEA,5455,4
...,...,...,...,...,...
5042,Meal,87,LATAM,14507,4
5043,Meal,87,LATAM,15625,4
5044,Meal,93,LATAM,5230,4
5045,Meal,93,LATAM,14942,4


# Task 4

So that you can take a more detailed look at the lowest performing hotels, you want to get service and branch information where the average rating for the branch and service combination is lower than 4.5 - the target set by management.  

- Your output should be a DataFrame named 'average_rating'
- It should return the `service_id` and `branch_id`, and the average rating (`avg_rating`)
- Values should be rounded to 2 decimal places where appropriate.

In [10]:
-- Write your query for task 4 in this cell

SELECT service_id, branch_id, ROUND (AVG (rating), 2) AS avg_rating
FROM public.request
GROUP BY service_id, branch_id
HAVING AVG (rating) < 4.5
ORDER BY avg_rating;

,service_id,branch_id,avg_rating
0,4,20,3.00
1,4,39,3.00
2,2,100,3.00
3,2,99,3.29
4,3,8,3.38
...,...,...,...
210,4,24,4.00
211,4,34,4.33
212,1,90,4.38
213,4,54,4.40
